In [2]:
import librosa

# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os

audioFilePaths = []
recordingInfoFilePaths = []
for dirname, _, filenames in os.walk('C:/Users/u144572/self_development/mgr/Respiratory_Sound_Database/audio_and_txt_files'):
    for filename in filenames:
        fullPath = os.path.join(dirname, filename)
        if filename.endswith("wav"):
            audioFilePaths.append(fullPath)
        elif filename.endswith("txt"):
            recordingInfoFilePaths.append(fullPath) 
        #print(os.path.join(dirname, filename))

In [3]:
import csv

def extractFilenameWithoutExtension(fullPath):
    fileName = os.path.basename(fullPath)
    fileNameWithoutPath = os.path.splitext(fileName)[0]
    return fileNameWithoutPath

#(fullPaths:string[]) => dict<filename:string, (start, end, hasCrackles, hasWheezes)[]>
def getFileInfoDict(pathList):
    fileInfoDict = {}
    for path in pathList:
        fileNameWithoutPath = extractFilenameWithoutExtension(path) 
        infoList = []
        with open(path, 'r') as hFile:
            reader = csv.reader(hFile, delimiter='\t')
            for row in reader:
                startTime = float(row[0])
                endTime = float(row[1])
                hasCrackles = True if int(row[2]) == 1 else False
                hasWheezes = True if int(row[3]) == 1 else False
                infoList.append((startTime, endTime, hasCrackles, hasWheezes))
                
        fileInfoDict[fileNameWithoutPath] = infoList
    return fileInfoDict
 
audioFilePathsWithoutExtension = [extractFilenameWithoutExtension(path) for path in audioFilePaths]
fileInfoDict = getFileInfoDict(recordingInfoFilePaths)
#List order is aligned with the order in which the audio clips are extracted
fileInfoList = [fileInfoDict[audioPath] for audioPath in audioFilePathsWithoutExtension] 

In [4]:
sec_sum_of_audio_files = 0
for x in fileInfoDict:
    sec_sum_of_audio_files += librosa.get_duration(path="C:/Users/u144572/self_development/mgr/Respiratory_Sound_Database/audio_and_txt_files/" + x + ".wav")

print(10089/(sec_sum_of_audio_files/60))

30.614427272538645


In [18]:
deases_dict = {}
for x in fileInfoDict:
    deases_dict[x] = {
            "crackles": False, 
            "wheezes": False, 
            "cycles_no": len(fileInfoDict[x]),
            "audio_length": librosa.get_duration(path="C:/Users/u144572/self_development/mgr/Respiratory_Sound_Database/audio_and_txt_files/" + x + ".wav"),
            "rr": round(len(fileInfoDict[x])/(librosa.get_duration(path="C:/Users/u144572/self_development/mgr/Respiratory_Sound_Database/audio_and_txt_files/" + x + ".wav")/60),2)
    }
    for y in fileInfoDict[x]:
            if y[2] == 1:
                deases_dict[x]["crackles"] = True
            if y[3] == 1:
                deases_dict[x]["wheezes"] = True
                 

In [20]:
healthy_cycles_no = 0
healthy_length = 0

crackles_cycles_no = 0
crackles_length = 0

wheezes_cycles_no = 0
wheezes_length = 0

both_cycles_no = 0
both_length = 0

for x in deases_dict:
    if deases_dict[x]["crackles"] == True and deases_dict[x]["wheezes"] == True:
        both_cycles_no += deases_dict[x]["cycles_no"]
        both_length += deases_dict[x]["audio_length"]
    if deases_dict[x]["crackles"] == False and deases_dict[x]["wheezes"] == True:
        wheezes_cycles_no += deases_dict[x]["cycles_no"]
        wheezes_length += deases_dict[x]["audio_length"]
    if deases_dict[x]["crackles"] == True and deases_dict[x]["wheezes"] == False:
        crackles_cycles_no += deases_dict[x]["cycles_no"]
        crackles_length += deases_dict[x]["audio_length"]
    else:
        healthy_cycles_no += deases_dict[x]["cycles_no"]
        healthy_length += deases_dict[x]["audio_length"]

healthy_rr = healthy_cycles_no/(healthy_length/60)
crackles_rr = crackles_cycles_no/(crackles_length/60)
wheezes_rr = wheezes_cycles_no/(wheezes_length/60)
both_rr = both_cycles_no/(both_length/60)
print(healthy_rr, crackles_rr, wheezes_rr, both_rr)

21.22515990207822 20.333277580864706 21.628346211101015 20.964553424222725


In [21]:
deases_dict["226_1b1_Al_sc_Meditron"]

{'crackles': True,
 'wheezes': False,
 'cycles_no': 10,
 'audio_length': 20.0,
 'rr': 30.0}